## Query ads datas from TIKTOK and save the data as json file to local

In [5]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
access_token = os.getenv('TOKEN')

def make_api_call(access_token, url, headers, payload):
    has_more = True
    all_results = []

    while has_more and len(all_results) < 2000:
        # Make the API request
        response = requests.post(f'{url}?fields={fields}', headers=headers, json=payload)

        # Check the response status
        if response.status_code == 200:
            # The request was successful
            data = response.json()

            # Check if there are more results
            query_data = data.get('data', {})
            has_more = query_data.get('has_more', False)

            # If there are more results, update the payload for the next request
            if has_more:
                payload['search_id'] = query_data.get('search_id', '')

            # Append the data to the results
            all_results.extend(data.get('data', {}).get('ads', []))
        else:
            # Handle the error
            print(f"Error: {response.status_code}, {response.text}")
            break  # exit the loop on error

    return all_results

url = 'https://open.tiktokapis.com/v2/research/adlib/ad/query/'
fields = 'ad.id,ad.first_shown_date,ad.last_shown_date,ad.status,ad.status_statement,ad.videos,ad.image_urls,ad.reach,advertiser.business_id,advertiser.business_name,advertiser.paid_for_by'

# Request headers
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}
coun_codes = ["AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", "GR", "HU", "IE", "IT", "LV", "LT", "LU", "MT", "NL", "PL", "PT", "RO", "SK", "SI", "ES", "SE", "NO", "IS", "LI", "GB", "CH"]

# Request payload
payload = {
    "fields": "ad.id,ad.first_shown_date,ad.last_shown_date,ad.status,ad.status_statement,ad.videos,ad.image_urls,ad.reach,advertiser.business_id,advertiser.business_name,advertiser.paid_for_by",
    "filters": {
        "ad_published_date_range": {
            "min": "20230101",
            "max": "20231121"
        },
        "country_codes": coun_codes, 
    },
    "max_count": 50
}

# Make the API call
all_results = make_api_call(access_token, url, headers, payload)

In [6]:
import json
json_filename = 'tiktok_ads_data.json'

# Write the data to JSON
with open(json_filename, 'w') as jsonfile:
    json.dump(all_results, jsonfile, indent=2)

print(f'Data has been saved to {json_filename}')

Data has been saved to tiktok_ads_data.json


## Testing ads details query

In [11]:
from pprint import pprint
# Set the ad ID you want to get details for
ad_id = 1776307088392242

# API endpoint for getting ad details
url = 'https://open.tiktokapis.com/v2/research/adlib/ad/detail/'

# Request headers
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Request payload
payload = {
    'fields': 'ad.id,ad.reach,advertiser.follower_count,advertiser.profile_url,ad_group.targeting_info',
    'ad_id': ad_id
}

# Make the API request
response = requests.post(f'{url}?fields={payload["fields"]}', headers=headers, json=payload)

# Check the response status
if response.status_code == 200:
    # The request was successful
    data = response.json()
    
    # Print or process the ad details as needed
    pprint(data)
else:
    # Handle the error
    print(f"Error: {response.status_code}, {response.text}")

{'data': {'ad': {'id': 1776307088392242,
                 'reach': {'unique_users_seen': '10M-20M',
                           'unique_users_seen_by_country': {'AT': '166K',
                                                            'BE': '490K',
                                                            'CZ': '1.3M',
                                                            'DK': '38K',
                                                            'ES': '6.9M',
                                                            'FI': '478K',
                                                            'IE': '121K',
                                                            'IT': '8.0M',
                                                            'NL': '165K',
                                                            'SE': '60K'}}},
          'ad_group': {'targeting_info': {'age': {'13-17': True,
                                                  '18-24': True,
                             

## Since TikTok has the limitation of 1000 request/day for API call, we only loop through the first 1000 ads ids to get ads details.

In [19]:

# Load ad IDs from the JSON file
with open('tiktok_ads_data.json', 'r') as file:
    ad_data = json.load(file)

# API endpoint for getting ad details
url = 'https://open.tiktokapis.com/v2/research/adlib/ad/detail/'

# Request headers
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# List to store results of ad detail queries
ad_detail_results = []

# Loop through the first five ad data entries
for ad_data_entry in ad_data[:1000]:
    ad_id = ad_data_entry['ad']['id']

    # Request payload
    payload = {
        'fields': 'ad.id,ad.reach,advertiser.follower_count,advertiser.profile_url,ad_group.targeting_info',
        'ad_id': ad_id
    }

    # Make the API request
    response = requests.post(f'{url}?fields={payload["fields"]}', headers=headers, json=payload)

    # Check the response status
    if response.status_code == 200:
        # The request was successful
        data = response.json().get('data', {})
        ad_detail_results.append(data)
    else:
        # Handle the error
        print(f"Error for ad ID {ad_id}: {response.status_code}, {response.text}")

# Now ad_detail_results contains the results of ad detail queries for the first five ad IDs in the JSON file
pprint(ad_detail_results)


[{'ad': {'id': 1776307088392242,
         'reach': {'unique_users_seen': '10M-20M',
                   'unique_users_seen_by_country': {'AT': '166K',
                                                    'BE': '490K',
                                                    'CZ': '1.3M',
                                                    'DK': '38K',
                                                    'ES': '6.9M',
                                                    'FI': '478K',
                                                    'IE': '121K',
                                                    'IT': '8.0M',
                                                    'NL': '165K',
                                                    'SE': '60K'}}},
  'ad_group': {'targeting_info': {'age': {'13-17': True,
                                          '18-24': True,
                                          '25-34': True,
                                          '35-44': True,
                           

In [20]:
import json
json_filename = 'ads_details.json'

# Write the data to JSON
with open(json_filename, 'w') as jsonfile:
    json.dump(ad_detail_results, jsonfile, indent=2)

print(f'Data has been saved to {json_filename}')

Data has been saved to ads_details.json


## Since TIKTOK API has limitation of 1000 request/day, run the code for geting ads details again to get more data

In [26]:

# Load ad IDs from the JSON file
with open('tiktok_ads_data.json', 'r') as file:
    ad_data = json.load(file)

# API endpoint for getting ad details
url = 'https://open.tiktokapis.com/v2/research/adlib/ad/detail/'

# Request headers
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# List to store results of ad detail queries
ad_detail_results2 = []

# Loop through the first five ad data entries
for ad_data_entry in ad_data[1001:2000]:
    ad_id = ad_data_entry['ad']['id']

    # Request payload
    payload = {
        'fields': 'ad.id,ad.reach,advertiser.follower_count,advertiser.profile_url,ad_group.targeting_info',
        'ad_id': ad_id
    }

    # Make the API request
    response = requests.post(f'{url}?fields={payload["fields"]}', headers=headers, json=payload)

    # Check the response status
    if response.status_code == 200:
        # The request was successful
        data = response.json().get('data', {})
        ad_detail_results2.append(data)
    else:
        # Handle the error
        print(f"Error for ad ID {ad_id}: {response.status_code}, {response.text}")

# Now ad_detail_results contains the results of ad detail queries for the first five ad IDs in the JSON file
pprint(ad_detail_results2)

Error for ad ID 1765674190951425: 500, {"error":{"code":"internal_error","message":"Something is wrong. Please try again later.","log_id":"20231127032939EF516B78478371AC6329"}}
Error for ad ID 1781458693482498: 429, {"data":{},"error":{"code":"rate_limit_exceeded","message":"The API rate limit exceeded. Please try again later.","log_id":"20231127033309269902844C5004D9DE5C"}}
Error for ad ID 1763411733431313: 503, <HTML><HEAD>
<TITLE>Service Unavailable</TITLE>
</HEAD><BODY>
<H1>Service Unavailable - Zero size object</H1>
The server is temporarily unable to service your request.  Please try again
later.<P>
Reference&#32;&#35;15&#46;d644c717&#46;1701056327&#46;4ce76b90
</BODY></HTML>

[{'ad': {'id': 1766153523550226,
         'reach': {'unique_users_seen': '1M-10M',
                   'unique_users_seen_by_country': {'GB': '5.6M'}}},
  'ad_group': {'targeting_info': {'age': {'13-17': False,
                                          '18-24': True,
                                         

In [28]:

# Load existing ad details from the JSON file
with open('ads_details.json', 'r') as file:
    existing_data = json.load(file)

# Append new data to existing data
existing_data.extend(ad_detail_results2)

# Write the updated data to the JSON file
with open('ads_details.json', 'w') as file:
    json.dump(existing_data, file, indent=2)


## Merge tiktok_ads_data.csv and ads_details.csv

In [ ]:
import pandas as pd
def json_to_csv(json_file, csv_file):
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Convert to DataFrame
    df = pd.json_normalize(data)

    # Save to CSV
    df.to_csv(csv_file, index=False)

# Convert json to csv
json_to_csv('ads_details.json', 'ads_details.csv')
json_to_csv('tiktok_ads_data.json', 'tiktok_ads_data.csv')

In [ ]:
# Check tiktok_ads_data csv file
df_ads = pd.read_csv("tiktok_ads_data.csv")
df_ads.head()

In [ ]:
# Check ads_details csv file
df_details = pd.read_csv("ads_details.csv")
df_details.head()

In [ ]:
# Check file columns
df_details.columns

In [ ]:
df_ads.columns

In [ ]:
# Rename the primary key column's name
df_details.rename(columns={'ad.id': 'ad.id'}, inplace=True)

# Merge the DataFrames
merged_df = pd.merge(df_ads, df_details, on='ad.id', how='inner')

# Drop rows where 'ad.id' is null
merged_df = merged_df.dropna(subset=['ad.id'])
merged_df

In [ ]:
# Convert merged_df to csv file
merged_df.to_csv('merged_data.csv', index=False)